### Analyizng Used Car Sales on Ebay.de

In this project we will clean and analyze a data set which represents used car listings that have been scrapped from [classifieds](https://www.ebay.de/b/Automobile/9801/bn_1845328) section of the German eBay website.  
Our data set contains 50,000 data points from the full dataset, picked at random. This subset of the full data set should be representative enough to provide accurate insights.

The data dictionary provided with data is as follows:

* `dateCrawled` - When this ad was first crawled. All field-values are taken from this date.
* `name` - Name of the car.
* `seller` - Whether the seller is private or a dealer.
* `offerType` - The type of listing
* `price` - The price on the ad to sell the car.
* `abtest` - Whether the listing is included in an A/B test.
* `vehicleType` - The vehicle Type.
* `yearOfRegistration` - The year in which the car was first registered.
* `gearbox` - The transmission type.
* `powerPS` - The power of the car in PS.
* `model` - The car model name.
* `kilometer` - How many kilometers the car has driven.
* `monthOfRegistration` - The month in which the car was first registered.
* `fuelType` - What type of fuel the car uses.
* `brand` - The brand of the car.
* `notRepairedDamage` - If the car has a damage which is not yet repaired.
* `dateCreated` - The date on which the eBay listing was created.
* `nrOfPictures` - The number of pictures in the ad.
* `postalCode` - The postal code for the location of the vehicle.
* `lastSeenOnline` - When the crawler saw this ad last online.

In [1]:
import numpy as np
import pandas as pd

autos = pd.read_csv('autos.csv', encoding = 'Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Pandas dataframe info() method provides a very usefull summary of all the objects contained within the data set, including the index dtype and column dtypes, non-null values and memory usage.

Based on the output above we can see that majority of the columns making up our data set are of the object type which represents `string` in the python data types. This includes columns which we would expect to see under different data types, such as price, odometer or dateCreated. Also, number of columns in our data have missing values (count of non-null values less than 50,000). It is worth noting that none of the columns have more than 20% of nulls.

In [2]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Next step on the list is to rename some of the existing column names. This cosmetic change will make the header a little bit more readable and easier to work with.

In [3]:
autos.rename(columns={"yearOfRegistration":"registration_year","monthOfRegistration":"registration_month","notRepairedDamage":"unrepaired_damage","dateCreated":"ad_created",'dateCrawled':'date_crawled','lastSeen':'last_seen'},inplace = True)

In [4]:
autos.head()

,date_crawled,name,seller,offerType,price,abtest,vehicleType,registration_year,gearbox,powerPS,model,odometer,registration_month,fuelType,brand,unrepaired_damage,ad_created,nrOfPictures,postalCode,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Data Cleansing and Exploration

Based on the initial research into the data we found that some of the columns might be redundant by holding identical or almost identical data in all of the records. Some other columns have numbers stored as text and those need to be converted back to numbers.

In [5]:
autos.describe(include='all')

,date_crawled,name,seller,offerType,price,abtest,vehicleType,registration_year,gearbox,powerPS,model,odometer,registration_month,fuelType,brand,unrepaired_damage,ad_created,nrOfPictures,postalCode,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-21 20:37:19,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Straight away the frequency metric reveals that there are two columns where almost all values are identical. Those columns are:
* **seller**
* **offerType**

Additionally, column called **nrOfPictures** looks suspicious by having returned nothing or zeros for all of the statistics generated by the `describe` method. Let's investigate the nrOfPictures column and if it contains unsuable data we can drop it along with two other redundant columns.

In [6]:
autos['nrOfPictures'].value_counts(dropna=False)

0    50000
Name: nrOfPictures, dtype: int64

The only value contained in the `nrOfPiuctures` column is zero. Let's drop the redundant columns before moving on.

In [7]:
autos.drop(columns=['seller','offerType'], inplace=True)

Finally, we can see that number of columns have numerical data stored as text. Those columns are as follows:
* price
* odometer

Both of these column contains numerical values concatenated with non-numerical characters therefore we will have to remove the latter ones first before converting both columns to the correct data type.

In [8]:
print(autos['price'].unique())
print("\n")
print(autos['odometer'].unique())

['$5,000' '$8,500' '$8,990' ... '$385' '$22,200' '$16,995']


['150,000km' '70,000km' '50,000km' '80,000km' '10,000km' '30,000km'
 '125,000km' '90,000km' '20,000km' '60,000km' '5,000km' '100,000km'
 '40,000km']


Let's remove those non-numerical characters and convert both columns to numeric data type.
        
    

In [9]:
autos['price'] = autos['price'].str.replace(',','').str.replace('$','').astype(int)
autos['price'].head()

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [10]:
autos['odometer'] = autos['odometer'].str.replace(',','').str.replace('km','').astype(int)
autos.rename(columns={"odometer":"odometer_km"},inplace=True)
autos['odometer_km'].head()

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: int64

### Deep dive into data analysis: price and odometer data

In [11]:
print('Number of unique values under the price header:',autos['price'].unique().shape[0])

Number of unique values under the price header: 2357


In [12]:
autos['price'].value_counts().sort_index(ascending=True).head(10)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
Name: price, dtype: int64

In [13]:
autos['price'].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [14]:
autos = autos[autos['price'].between(1,351000)]
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

A combination of `value_counts` method with sorting the output reveals that there are outliers hidden under the **`price`** header. There were 1421 listings with the price of 0 and on the other tip of the scale, there were few listings with the price of almost 1 million and over. While it is entirely possible that cars are sold for a very high price, there is a significant gap between those outliers and number of prices that have been incrementaly rising prior to reaching those sky high values.
Having removed the outliers from the data set, the statistics for the **`price`** column look much more realistic.  
The minimum price at which cars have been sold was 1 dollar and the top price was 350,000 dollars. On average, cars have been sold for a price of 5,888 US dollars.

In [15]:
print('Number of unique values under the odometer_km header:',autos['odometer_km'].unique().shape[0])

Number of unique values under the odometer_km header: 13


In [16]:
autos['odometer_km'].value_counts().sort_index(ascending=True)

5000        836
10000       253
20000       762
30000       780
40000       815
50000      1012
60000      1155
70000      1217
80000      1415
90000      1734
100000     2115
125000     5057
150000    31414
Name: odometer_km, dtype: int64

Analysis of the **`odometer_km`** column is very straighforward. There are only 13 unique values returned by that header and they are all rounded values which suggests that the values in that column were predfined for the end users in a way of a dropdown list or similar feature. Also, it is easy to spot, without running any statistics on that column, that majority of cars sold were the ones with higher milage (over 100,000 km).

### Exploring the date type columns

There are 5 columns that represent dates in our data set. There are 2 of them that have been added by the web crawler and the remianing three came with the data.

* `date_crawled`: added by the crawler
* `last_seen`: added by the crawler
* `ad_created`: from the website
* `registration_month`: from the website
* `registration_year`: from the website

All three dates added by the crawler have been recorded as strings therefore in order to enable us to make any analysis of those dates, we will have to convert them to numeric values. The two date type columns that came with the data have been saved as numeric values therefore we won't have to do any additional procressing on those columns in order to understand the distirbution better.

In [17]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


All three dates saved as string have been saved in the full timestamp format. The good news is that first 10 characters from each columns represent the dates and we can easily extract them to allow for analysis of the distribution within each column.

In [18]:
(autos['date_crawled']
    .str[:10]
    .value_counts(normalize = True,dropna=False)
    .sort_index(ascending = True))

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

Distribution of dates representing the **`date_crawled`** column suggest that the crawler was harvesting data daily for a period of just over a month. Each day's yield was uniform across the whole period.  

In [19]:
(autos['last_seen']
    .str[:10]
    .value_counts(normalize = True,dropna=False)
    .sort_index(ascending = True))

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

Distribution of values returned by the **`last_seen`** column paints similar picture with majority of dates returning similar yield except for the last 3 days where we can see significant spike in the percentage values (5 to 6 times the normal amout). Since the **`last_seen`** column represents the date for when the crawler has seen the ad for the last time, this might indicate that the spike associated with the last three days is to do with the fact that those were the last days when the crawler was active.

In [20]:
(autos['ad_created']
    .str[:10]
    .value_counts(normalize = True,dropna=False)
    .sort_index(ascending = True))

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

In [21]:
(autos['ad_created']
    .str[:10]
    .value_counts(normalize = False,dropna=False)
    .sort_index(ascending = True))

2015-06-11       1
2015-08-10       1
2015-09-09       1
2015-11-10       1
2015-12-05       1
2015-12-30       1
2016-01-03       1
2016-01-07       1
2016-01-10       2
2016-01-13       1
2016-01-14       1
2016-01-16       1
2016-01-22       1
2016-01-27       3
2016-01-29       1
2016-02-01       1
2016-02-02       2
2016-02-05       2
2016-02-07       1
2016-02-08       1
2016-02-09       1
2016-02-11       1
2016-02-12       2
2016-02-14       2
2016-02-16       1
2016-02-17       1
2016-02-18       2
2016-02-19       3
2016-02-20       2
2016-02-21       3
              ... 
2016-03-09    1610
2016-03-10    1549
2016-03-11    1598
2016-03-12    1785
2016-03-13     826
2016-03-14    1709
2016-03-15    1652
2016-03-16    1463
2016-03-17    1519
2016-03-18     660
2016-03-19    1636
2016-03-20    1843
2016-03-21    1825
2016-03-22    1593
2016-03-23    1557
2016-03-24    1422
2016-03-25    1542
2016-03-26    1567
2016-03-27    1505
2016-03-28    1699
2016-03-29    1653
2016-03-30  

Analysis of the **`ad_created`** column revels that most of the ads were created in a uniform manner in the same period when the crawler was active, yielding between 1400-1800 ads per day. This volume drops in the last 3 days which corresponds with the declining activity in the same three days as represented by the **`date_crawled`** column. Additionally, there was a very small volume of older ads, that have been creater prior to the crawling activity period including few ads created in 2015.

In [22]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Analysis of the **`registration_year`** column reveals that some of the data in this column is incorrect. It is not possible that any cars sold have been registered in year 1000 just like it is not possible that other cars have been registered in the future (year 9999). Since all of the cars have been listed on eBay in 2016, none of them should have the registration year newer than 2016. Similarly, since most of the modern cars have been produces in the last few decades, we should not see any cars registered prior to 1900.

In [23]:
autos['registration_year'].value_counts().sort_index(ascending=True)

1000       1
1001       1
1111       1
1800       2
1910       5
1927       1
1929       1
1931       1
1934       2
1937       4
1938       1
1939       1
1941       2
1943       1
1948       1
1950       3
1951       2
1952       1
1953       1
1954       2
1955       2
1956       4
1957       2
1958       4
1959       6
1960      23
1961       6
1962       4
1963       8
1964      12
        ... 
2000    3156
2001    2636
2002    2486
2003    2699
2004    2703
2005    2936
2006    2670
2007    2277
2008    2215
2009    2085
2010    1589
2011    1623
2012    1310
2013     803
2014     663
2015     392
2016    1220
2017    1392
2018     470
2019       2
2800       1
4100       1
4500       1
4800       1
5000       4
5911       1
6200       1
8888       1
9000       1
9999       3
Name: registration_year, Length: 95, dtype: int64

In [24]:
print('Percentage of cars register outside expected values:',autos[~autos['registration_year'].between(1900,2016)].shape[0] / autos.shape[0])

Percentage of cars register outside expected values: 0.038793369710697


Nearly 4% of cars have been registered with unexpected values therefore it is a good idea to remove them before another analysis of the distribution of the data.

In [25]:
autos = autos[autos['registration_year'].between(1900,2016)]
autos['registration_year'].value_counts(normalize=True).sort_values(ascending=False).head(20)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
2008    0.047450
2009    0.044665
1997    0.041794
2011    0.034768
2010    0.034040
1996    0.029412
2012    0.028063
1995    0.026285
2016    0.026135
2013    0.017202
Name: registration_year, dtype: float64

Based on the normalized distribution we can deduct that most cars have been registered in the last 20 years.

### Exploring Brands

In [26]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.211264
bmw               0.110045
opel              0.107581
mercedes_benz     0.096463
audi              0.086566
ford              0.069900
renault           0.047150
peugeot           0.029841
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008762
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

To analyze the brands we are going to select those brands of which cars represent at least 5% of all listings included in the data set. Unsurprisingly, the German brands made the top 5 brands and those 5 brands constitue 60% of all listings under our analysis. Let's dive deeper into the data and determine the price distribution wihin each brand.

In [27]:
brand_bool = autos['brand'].value_counts(normalize=True) >= .05
filtered_brands = brand_bool[brand_bool==True].index
print(filtered_brands)


Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford'], dtype='object')


In [28]:
brand_mean_prices = dict()
for i in filtered_brands:
    brand_bool = autos['brand'] == i
    mean_price = autos.loc[brand_bool,'price'].mean()
    brand_mean_prices[i] = int(mean_price)
print(brand_mean_prices)


{'ford': 3749, 'bmw': 8332, 'audi': 9336, 'volkswagen': 5402, 'opel': 2975, 'mercedes_benz': 8628}


It is clear from the above result that on average, cars belonging to the audi brand attracted the highest bids while opel lands on the opposite end of the spectrum. Volkswagen has been historically deemed as a quality car manufacturer therefore its attractive prices (mean price in the middle of the spectrum) explain why it landed a top spot in terms of popularity among all brands (21% share of all listings) 

### Exploring mileage

In [29]:
brand_mean_mileage = dict()
for i in filtered_brands:
    brand_bool = autos['brand'] == i
    mean_milleage = autos.loc[brand_bool,'odometer_km'].mean()
    brand_mean_mileage[i] = int(mean_milleage)
print(brand_mean_mileage)

{'ford': 124266, 'bmw': 132572, 'audi': 129157, 'volkswagen': 128707, 'opel': 129310, 'mercedes_benz': 130788}


In [30]:
bmp_series = pd.Series(brand_mean_prices)
print(bmp_series)
df = pd.DataFrame(bmp_series,columns=['mean_price'])
df

audi             9336
bmw              8332
ford             3749
mercedes_benz    8628
opel             2975
volkswagen       5402
dtype: int64


,mean_price
audi,9336
bmw,8332
ford,3749
mercedes_benz,8628
opel,2975
volkswagen,5402


In [31]:
bmm_series = pd.Series(brand_mean_mileage)
print(bmm_series)
df['mean_mileage'] = bmm_series
df

audi             129157
bmw              132572
ford             124266
mercedes_benz    130788
opel             129310
volkswagen       128707
dtype: int64


,mean_price,mean_mileage
audi,9336,129157
bmw,8332,132572
ford,3749,124266
mercedes_benz,8628,130788
opel,2975,129310
volkswagen,5402,128707


In this paragraph we managed to combine dictionaries with loops in order to come up with a mean values for price and mileage data associated with the top 6 brands. Subsequently, both dictionaries have been turned into Series structure and added to new Data Frame structure, which holds brand names as index and both dictionary values as columns.

Based on the data contained within this new data frame we can conclude that mileage does not have any significant affect on the average price for each brand since all brands we investigated yield very similar mean value for mileage.